# 20대 제안자 칼럼 생성

- 발의자

In [13]:
import pandas as pd 
import numpy as np 
from ast import literal_eval
from tqdm import tqdm 
import numpy as np

In [16]:
gender = pd.read_csv('20th-gender-bill-wo-glass.csv', index_col=0, encoding='utf-8')
print(gender.shape)
gender.head(3)

(1789, 19)


,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자,발의자,발의자_한글,제안정당,제안자_한글,성별,여성의원 수
0,2024984,처,일제하 일본군위안부 피해자에 대한 보호ㆍ지원 및 기념사업 등에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20 00:00:00,원안가결,,의안원문,제20대 (2016~2020) 제378회,대안의 생활안정지원대상자에게 「의료급여법」에 따른 본인 부담금 및 비급여비용에...,여성가족위원회,NaN,위원장,NaN,위원장,[],,0
1,2024980,처,양육비 이행확보 및 지원에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20 00:00:00,원안가결,,의안원문,제20대 (2016~2020) 제378회,대안의 한시적 양육비 긴급지원이 이루어진 경우 양육비 채무자에게 국세 체납처분...,여성가족위원회,NaN,위원장,NaN,위원장,[],,0
2,2024979,처,아동·청소년의 성보호에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20 00:00:00,원안가결,,의안원문,제20대 (2016~2020) 제378회,2. 대안의 아동？청소년 음란물 범죄의 규모와 형태가 갈수록 교묘해지고 있지만...,여성가족위원회,NaN,위원장,NaN,위원장,[],,0


In [17]:
# 정부, 위원장 발의일 때 
gender.loc[gender['제안자구분'] == '정부', '발의자_한글'] = '정부'
gender.loc[gender['제안자구분'] == '위원장', '발의자_한글'] = '위원장'

In [18]:
gender = gender[['의안번호','제안자_한글']]
gender.head(3)

,의안번호,제안자_한글
0,2024984,[]
1,2024980,[]
2,2024979,[]


In [19]:
%time gender['제안자_한글'] = gender['제안자_한글'].apply(literal_eval)

CPU times: user 93.1 ms, sys: 4.73 ms, total: 97.8 ms
Wall time: 98.4 ms


## 발의자 코딩

In [6]:
proposer_list = []

for names in gender['제안자_한글']: 
    proposer_set = set(proposer_list)
    names_set = set(names)
    minus = list(names_set.difference(proposer_set))
    proposer_list.extend(minus)
    
len(proposer_list)

317

In [7]:
def get_proposer_dummies(proposer): 
    for n in tqdm(range(proposer.shape[1])): 
        df = pd.get_dummies(proposer[n])
        if n == 0: 
            final_df = df 
        else: 
            final_df = final_df.add(df, fill_value=0)
    return final_df

In [8]:
proposer = gender['제안자_한글'].apply(lambda x: pd.Series(x))

%time proposer_dummies = get_proposer_dummies(proposer)
print(proposer_dummies.shape)

<ipython-input-8-c684986f93c6>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  proposer = gender['제안자_한글'].apply(lambda x: pd.Series(x))
100%|██████████| 121/121 [02:11<00:00,  1.09s/it]

CPU times: user 2min 8s, sys: 1.05 s, total: 2min 9s
Wall time: 2min 12s
(2016, 317)


In [9]:
proposer_dummies.head(3)

,,강길부,강병원,강석진,강석호,강창일,강효상,강훈식,경대수,고용진,...,홍문종,홍문표,홍영표,홍의락,홍익표,홍일표,홍철호,황영철,황주홍,황희
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
gender = pd.concat([gender, proposer_dummies], axis=1)

In [11]:
gender = gender.drop(['제안자_한글','의안번호'],1)

In [12]:
gender.head()

,,강길부,강병원,강석진,강석호,강창일,강효상,강훈식,경대수,고용진,...,홍문종,홍문표,홍영표,홍의락,홍익표,홍일표,홍철호,황영철,황주홍,황희
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
len(gender.columns)

317

In [14]:
gender.to_csv('20th-proposer-list-wo-glass.csv')

## 성별 평균 비교

### 여성

In [1]:
women_list = ['추미애','박영선','유승희','김영주','김상희','김현미','전혜숙',
             '서영교','인재근','한정애','전현희','남인순','진선미','유은혜',
              '백혜련','박경미','송옥주','이재정','문미옥','제윤경','권미혁',
             '정춘숙','정은혜','허윤정','나경원','이혜훈','박순자','박인숙',
             '이언주','김정재','송희경','임이자','최연혜','신보라','전희경',
             '김승희','윤종필','김순례','김현아','조배숙','박선숙','신용현',
             '박주현','김수민','김삼화','장정숙','최도자','심상정','이정미',
             '추혜선','권은희','손혜원','이은재']
no_list_women = []

In [2]:
len(women_list)

53

In [21]:
women = gender[women_list]
women.head()

KeyError: "None of [Index(['추미애', '박영선', '유승희', '김영주', '김상희', '김현미', '전혜숙', '서영교', '인재근', '한정애',\n       '전현희', '남인순', '진선미', '유은혜', '백혜련', '박경미', '송옥주', '이재정', '문미옥', '제윤경',\n       '권미혁', '정춘숙', '정은혜', '허윤정', '나경원', '이혜훈', '박순자', '박인숙', '이언주', '김정재',\n       '송희경', '임이자', '최연혜', '신보라', '전희경', '김승희', '윤종필', '김순례', '김현아', '조배숙',\n       '박선숙', '신용현', '박주현', '김수민', '김삼화', '장정숙', '최도자', '심상정', '이정미', '추혜선',\n       '권은희', '손혜원', '이은재'],\n      dtype='object')] are in the [columns]"

In [17]:
add_list = [0 for n in range(len(no_list_women))]
print(add_list)
print(len(add_list))

[]
0


In [18]:
women_count = women.sum(axis=0).tolist() + add_list
print(len(women_count))

women_count = np.array(women_count)
print('평균 여성 발의:', np.mean(women_count))

53
평균 여성 발의: 117.71698113207547


### 남성

- 보궐선거, 재보궐선거 등 국회의원 수를 정확히 책정할 수 없음. 
- 따라서 당시 국회의원수를 기준으로 (국회의원수 - 여성의원수)로 책정함. 
- 동명이인 있어도 함께 세므로 평균을 내는 데에는 상관 없음.

In [19]:
men = gender.drop(women_list, 1)
men.head()

,,강길부,강병원,강석진,강석호,강창일,강효상,강훈식,경대수,고용진,...,홍문종,홍문표,홍영표,홍의락,홍익표,홍일표,홍철호,황영철,황주홍,황희
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
total_num = 300
total_men_num = total_num - len(women_list) - len(no_list_women)
print('총 남자의원 수:', total_men_num)

총 남자의원 수: 247


In [21]:
men_count = np.array(men.sum(axis=0).tolist())
np.sum(men_count)

16557.0

In [22]:
print('평균 남성발의:', np.sum(men_count) / total_men_num)

평균 남성발의: 67.03238866396761


## 정당 코딩

In [22]:
party = pd.read_csv('20th-gender-bill-wo-glass.csv', index_col=0, encoding='utf-8')
party.head()

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자,발의자,발의자_한글,제안정당,제안자_한글,성별,여성의원 수
0,2024984,처,일제하 일본군위안부 피해자에 대한 보호ㆍ지원 및 기념사업 등에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20 00:00:00,원안가결,,의안원문,제20대 (2016~2020) 제378회,대안의 생활안정지원대상자에게 「의료급여법」에 따른 본인 부담금 및 비급여비용에...,여성가족위원회,NaN,위원장,NaN,위원장,[],,0
1,2024980,처,양육비 이행확보 및 지원에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20 00:00:00,원안가결,,의안원문,제20대 (2016~2020) 제378회,대안의 한시적 양육비 긴급지원이 이루어진 경우 양육비 채무자에게 국세 체납처분...,여성가족위원회,NaN,위원장,NaN,위원장,[],,0
2,2024979,처,아동·청소년의 성보호에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20 00:00:00,원안가결,,의안원문,제20대 (2016~2020) 제378회,2. 대안의 아동？청소년 음란물 범죄의 규모와 형태가 갈수록 교묘해지고 있지만...,여성가족위원회,NaN,위원장,NaN,위원장,[],,0
3,2024978,처,가족친화 사회환경의 조성 촉진에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20 00:00:00,원안가결,,의안원문,제20대 (2016~2020) 제378회,대안의 가족친화 사회환경 조성 기본계획이 수립된 경우 여성가족부장관은 지체 없...,여성가족위원회,NaN,위원장,NaN,위원장,[],,0
4,2024977,처,가정폭력방지 및 피해자보호 등에 관한 법률 일부개정법률안,위원장,2020-05-20,2020-05-20 00:00:00,원안가결,,의안원문,제20대 (2016~2020) 제378회,대안의 배우자가 대한민국 국민이 아닌 외국인 가정폭력 피해자도 외국인보호시설의...,여성가족위원회,NaN,위원장,NaN,위원장,[],,0


In [23]:
party['제안정당'].unique()

array(['위원장', '더불어민주당', '미래통합당', '정부', '정의당', '자유한국당', '무소속', '바른미래당',
       '민주평화당', '민중당', '대한애국당', '국민의당', '바른정당', '새누리당', '대안신당', '미래한국당'],
      dtype=object)

In [24]:
# party.loc[party['제안정당'] == '민주자유당', '제안정당'] = '신한국당'
# party.loc[party['제안정당'] == '민주당', '제안정당'] = '통합민주당'

In [25]:
conservative = ['미래통합당','자유한국당','바른미래당','대한애국당','바른정당','새누리당','대안신당','미래한국당']
progressive = ['더불어민주당','정의당','민주평화당','민중당','국민의당']

party.loc[party['제안정당'].isin(conservative), '정당성향'] = '보수'
party.loc[party['제안정당'].isin(progressive), '정당성향'] = '진보'

In [26]:
party['정당성향'].unique()

array([nan, '진보', '보수'], dtype=object)

In [27]:
party['정당성향'].value_counts()

진보    1122
보수     500
Name: 정당성향, dtype: int64

In [28]:
party.to_csv('20th-gender-bill-v3-wo-glass.csv', encoding='utf-8-sig')